In [1]:
import pandas as pd

In [2]:
import numpy as np
from pandas.io.html import read_html

In [19]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
DFs = read_html(page, match='Borough', header=0) # a list of dataframes
df = DFs[0]#getting the dataframe frm the list

In [20]:
print('not assigned boroughs: ', df.loc[df.Borough == 'Not assigned', 'Borough'].count())
print('not assigned neighbourhoods: ', df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'].count())

not assigned boroughs:  77
not assigned neighbourhoods:  78


In [21]:
df.Borough.replace('Not assigned', np.nan, inplace=True)


In [22]:
df.dropna(subset=['Borough'], inplace=True)
print('not assigned boroughs: ', df.loc[df.Borough == 'Not assigned', 'Borough'].count())

not assigned boroughs:  0


In [23]:
df.set_index(['Postcode', 'Borough'], inplace=True)
d = df.groupby(level=['Postcode','Borough']).agg(','.join)
d.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [24]:
d.reset_index(level=['Borough'], inplace=True)
d.columns

Index(['Borough', 'Neighbourhood'], dtype='object')

In [25]:
d.loc['M1B']
print('not assigned neighbourhoods: ', d.loc[d.Neighbourhood == 'Not assigned', 'Neighbourhood'].count())
d.loc[d.Neighbourhood == 'Not assigned', 'Borough']

not assigned neighbourhoods:  1


Postcode
M7A    Queen's Park
Name: Borough, dtype: object

In [26]:
d.loc[d.Neighbourhood == 'Not assigned', 'Neighbourhood'] = 'Queen\'s park'
d.shape


(103, 2)

In [27]:
import geocoder

In [11]:
conda install -c conda-forge geocoder


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    future-0.18.2              |           py36_0         713 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         982 KB

The

In [28]:

latitude = []
longitude = []
coordinates = None
for code in df.Postcode:
    while (coordinates == None):
        coordinates = geocoder.google(code + ', Toronto, Ontario').latlng
    latitude.append(coordinates[0])
    longitude.append(coordinates[1])

AttributeError: 'DataFrame' object has no attribute 'Postcode'

In [29]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

FileNotFoundError: [Errno 2] File b'Geospatial_Coordinates.csv' does not exist: b'Geospatial_Coordinates.csv'

In [ ]:
d['Latitude'] = 0
d['Longitude'] = 0

In [ ]:
for code in d.index:
    if code in coordinates['Postal Code'].values:
        d.loc[code, 'Latitude'] = coordinates.loc[coordinates['Postal Code'] == code, 'Latitude'].values
        d.loc[code, 'Longitude'] = coordinates.loc[coordinates['Postal Code'] == code, 'Longitude'].values

In [ ]:
d.head(40)
